# AV2 - Suicide Statistics

## Overview

Aqui iremos treinar uma rede neural para classificar 

The quesion that we are looking at, is weather the geographical origin of songs can be predicted based on different audio features. The data can be downloaded [here](http://archive.ics.uci.edu/ml/datasets/geographical+original+of+music). Prior exploration has been performed on this data. The most well-known study can be viewed [here](http://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=7023456).

## Preparativos

In [1]:
# TensorFlow and tf.keras
import tensorflow as tf
import keras

# Helper libraries
import numpy as np
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

print(tf.__version__)

Using TensorFlow backend.


0.12.0


O Dataset agrupou dados de 1979 há 2016. Contendo seis Colunas: País, Ano, Idade, Agrupamento de Idades e Sexo.

In [34]:
print(os.listdir("./../data/"))

dataset = pd.read_csv('../data/who_suicide_statistics.csv')

print("\nNúmero de Rows:", dataset.shape[0])
print("Número de Colunas:", dataset.shape[1])

dataset.head()

['.DS_Store', 'who_suicide_statistics.csv']

Número de Rows: 43776
Número de Colunas: 6


,country,year,sex,age,suicides_no,population
0,Albania,1985,female,15-24 years,NaN,277900.0
1,Albania,1985,female,25-34 years,NaN,246800.0
2,Albania,1985,female,35-54 years,NaN,267500.0
3,Albania,1985,female,5-14 years,NaN,298300.0
4,Albania,1985,female,55-74 years,NaN,138700.0


O Dataset, baixado do kaggle, só nos oferece um arquivo CSV, iremos utilizar o metodo train_test_split() do sklearn para dividir o nosso csv. (iremos dividir 20% para teste e 80% para treino).

In [37]:
dataset.dropna(axis=0,inplace =True)
dataset.isnull().sum()

dataset['country'] = pd.factorize(dataset['country'])[0] + 1
dataset['sex'] = pd.factorize(dataset['sex'])[0] + 1
dataset['age'] = pd.factorize(dataset['age'])[0] + 1

y = dataset.year
X = dataset.drop('year', axis=1)

dataset.head()

,country,year,sex,age,suicides_no,population
24,1,1987,1,1,14.0,289700.0
25,1,1987,1,2,4.0,257200.0
26,1,1987,1,3,6.0,278800.0
27,1,1987,1,4,0.0,311000.0
28,1,1987,1,5,0.0,144600.0


In [38]:
import pandas as pd
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) # 0.2 = 20% of 100%

Logo abaixo está o resultado. E como dito, possuimos cinco Features: Country, Year, Sex, Age and Population.

In [39]:
print("Training set: {}".format(X_train.shape))  # 404 examples, 13 features
print("Testing set:  {}".format(X_test.shape))   # 102 examples, 13 features

Training set: (28848, 5)
Testing set:  (7212, 5)


In [32]:
def build_model():
  model = keras.Sequential([
    keras.layers.Dense(64, activation=tf.nn.relu,
                       input_shape=(X_train.shape[1],)),
    keras.layers.Dense(64, activation=tf.nn.relu),
    keras.layers.Dense(1)
  ])

  optimizer = tf.train.RMSPropOptimizer(0.001)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae'])
  return model

model = build_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 64)                384       
_________________________________________________________________
dense_14 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 65        
Total params: 4,609
Trainable params: 4,609
Non-trainable params: 0
_________________________________________________________________


In [41]:
from sklearn.linear_model import LinearRegression

# Display training progress by printing a single dot for each completed epoch
class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0: print('')
    print('.', end='')

EPOCHS = 500

history = model.fit(X_train, y_train, epochs=EPOCHS,
                    validation_split=0.2, verbose=0,
                    callbacks=[PrintDot()])


....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................